In [10]:
import tensorflow as tf
import os
import sys
import go_datafile_reader
import model
import numpy as np
reload(model)
reload(go_datafile_reader)

<module 'go_datafile_reader' from 'go_datafile_reader.pyc'>

In [2]:
data_dir = "/home/justin/Programming/GoAI/MovePredictionCNN/data/input/test"
ckpt_path = "/home/justin/Programming/GoAI/MovePredictionCNN/data/working/board_eval_cnn_5layer.ckpt"
RECORD_BYTES = 5

In [3]:

file_count = 0
data_files = []
for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            data_files.append(filepath)
            file_count+=1
print file_count

14873


In [4]:
reader = go_datafile_reader.GoDatafileReader(data_files)
batch_aggregator = go_datafile_reader.BatchAggregator(reader)

In [5]:
x, ownership = model.place_holders()
y_conv = model.model(x)
loss = model.loss_function(ownership, y_conv)

train_op = model.train_step(loss)


prediction = tf.round(y_conv)
correct_prediction = tf.equal(ownership, prediction)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())
saver.restore(sess, ckpt_path)

In [6]:
for k in range(100000):
    x_batch, y_batch = batch_aggregator.get_batch(50)
    _, loss_value, y_value = sess.run([train_op, loss, y_conv], feed_dict={x:x_batch, ownership:y_batch})
    if k % 10 == 0:
        acc = accuracy.eval(feed_dict={x:x_batch, ownership:y_batch})
        print loss_value, acc
    if k % 1000 == 0:
        print "Saving Checkpoint..."
        saver.save(sess, ckpt_path)
    

Got to end of mega batch, reinitializing...
2468.22 0.845152
Saving Checkpoint...
2270.2 0.855956
2440.9 0.843823
2654.3 0.83108
2831.8 0.814903
2537.17 0.835568
2536.57 0.837673
2597.35 0.833019
2656.67 0.832022
2295.22 0.851025
2773.33 0.827258
2262.35 0.853407
2496.81 0.841717
2610.25 0.829584
2386.69 0.846039
2589.55 0.831911
2172.32 0.866648
2469.6 0.83856


KeyboardInterrupt: 

In [9]:
x_batch, y_batch = batch_aggregator.get_batch(50)
y_val, y_preds = sess.run([y_conv, prediction], feed_dict={x:x_batch, ownership:y_batch})

In [16]:
idx = 5
y_pred = np.reshape(y_preds[idx], [19,19])
y_true = np.reshape(y_batch[idx], [19,19])
feature_cube = x_batch[idx]

for i in xrange(19):
    current_row = ""
    for j in xrange(19):
        b_sum = feature_cube[i][j][0] + feature_cube[i][j][1] + feature_cube[i][j][2]
        w_sum = feature_cube[i][j][3] + feature_cube[i][j][4] + feature_cube[i][j][5]
        if b_sum > 0:
            current_row += '1'
        elif w_sum > 0:
            current_row += '0'
        else:
            current_row += '*'
    current_row += "   "
    for j in xrange(19):
        if y_pred[i][j] == 1:
            current_row += '1'
        elif y_pred[i][j] == 0:
            current_row += '0'
        else:
            current_row += '*'
    current_row += "   "
    for j in xrange(19):
        if y_true[i][j] == 1:
            current_row += '1'
        elif y_true[i][j] == 0:
            current_row += '0'
        else:
            current_row += '*'
    print current_row


    

*******************   1111111101000000000   1111111110000000000
*******************   1111101111100000001   1111101111000000000
*111*1*************   0111010010000000111   0111000010000000001
*1000********0**1**   0100001111000001111   0100001111000000111
*00****************   0000010011000001111   0000011010000101111
**10***********1***   0010000011000011111   1000001011000111111
*************0*****   1100111001100011111   1100111000000011111
*******************   0110011110010011111   0110001010001011111
******1************   0011111111010011111   0011101111011111111
**0***100**********   0000111001001000111   0000111000001111111
******110*******1**   0000001101011011111   0000001100011111111
***0111011*********   0000111011111011111   0000111011111111101
***0100001*********   0000100001111101101   0000100001111111101
***00100*01********   0000010001111110000   0000010011111111101
*****1**011********   0000111001111100000   0000111001111111000
*****1**00011*0*0**   001111100001110000

In [86]:
old_index = 0
old_k = 0
for k in range(40000):
    final_state, _, feature_cube = reader.read_sample()
    if reader.index_of_file != old_index:
        print k-old_k
        old_k = k
        old_index = reader.index_of_file    

0
208
267
266
252
252
269
294
240
252
277
269
285
260
274
328
309
238
314
270
253
283
245
242
324
238
266
223
235
235
254
298
318
251
258
267
241
259
231
279
312
246
296
221
286
243
266
256
241
308
241
294
292
279
276
345
265
287
252
264
258
267
238
254
243
265
224
265
301
262
225
235
231
314
316
267
322
277
246
262
242
280
230
280
290
228
240
261
314
306
238
258
272
291
249
261
249
261
271
255
288
258
242
246
301
285
324
265
337
363
254
249
220
237
237
264
246
244
244
297
302
285
305
275
261
257
244
242
280
251
290
268
333
296
266
284
263
287
321
288
271
263
257
287
249
236
342
282
265
